In [14]:
from cryptography.fernet import Fernet

# Generate a key for encryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

data = "Suraj"
encoded_data = data.encode()
print(encoded_data)
print(encoded_data.decode())

encrypted_data = cipher_suite.encrypt(data.encode())
print(encrypted_data)

print(cipher_suite.decrypt(encrypted_data))

b'Suraj'
Suraj
b'gAAAAABmybANAny8vdyK7zqRgcupNO9mPldW9qF44hu9gt0pa6i2Rf8lRmRrJs5pyrLLdU48e4axJ7kgZ6eNzUwlpxqiIoYutA=='
b'Suraj'


In [7]:
from cryptography.fernet import Fernet

# Generate a key for encryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Function to encrypt data
def encrypt_data(data: str) -> str:
    encrypted_data = cipher_suite.encrypt(data.encode())
    return encrypted_data.decode()

# Function to decrypt data
def decrypt_data(encrypted_data: str) -> str:
    decrypted_data = cipher_suite.decrypt(encrypted_data.encode())
    return decrypted_data.decode()

# Example usage
if __name__ == "__main__":
    # Receive data from the user
    user_data = input("Enter the data you want to encrypt: ")

    # Encrypt the data
    encrypted = encrypt_data(user_data)
    print(f"Encrypted data: {encrypted}")

    user_decrypt = input("Do you want to decrypt the data you provided? (Yes | No) : ").lower()
    if user_decrypt == "yes":
        # Decrypt the data (for demonstration purposes)
        decrypted = decrypt_data(encrypted)
        print(f"Decrypted data: {decrypted}")
    elif user_decrypt == "no":
        print(f"Your data is still encryted. Encrypted Data : {encrypted}")


Encrypted data: gAAAAABmya3AQPiYdts00qpE8PNjSZUDohrSZ1qth3_5GOT4XY6IVo7syg86Fr6f-TdGvEybHm3Lk5OnnoDdNHjqY0JN0Nb9-aF44ubDSq5r1ipGqsYuYxo=
Decrypted data: sharmasuraj1303@gmail.com


### Try HKAC with the below given code

In [17]:
from cryptography.fernet import Fernet

data_store = {}

# Dictionary to store encrypted data and their corresponding keys
# Function to encrypt data
def encrypt_data(data: str) -> (str, str):
    key = Fernet.generate_key()
    cipher_suite = Fernet(key)
    encrypted_data = cipher_suite.encrypt(data.encode())
    return encrypted_data.decode(), key.decode()

# Function to decrypt data
def decrypt_data(encrypted_data: str, key: str) -> str:
    cipher_suite = Fernet(key.encode())
    decrypted_data = cipher_suite.decrypt(encrypted_data.encode())
    return decrypted_data.decode()

# Function to store data
def store_data(identifier: str, encrypted_data: str, key: str):
    data_store[identifier] = {'encrypted_data': encrypted_data, 'key': key}

# Function to retrieve and decrypt data
def retrieve_and_decrypt(identifier: str) -> str:
    if identifier in data_store:
        encrypted_data = data_store[identifier]['encrypted_data']
        key = data_store[identifier]['key']
        return decrypt_data(encrypted_data, key)
    else:
        return "No such identifier found."

# Main function to handle user interaction
def main():
    while True:
        choice = input("Do you want to (E)ncrypt new data, (D)ecrypt existing data, or (Q)uit? ").upper()
        
        if choice == 'E':
            identifier = input("Enter an identifier for the data (e.g., 'email', 'password'): ")
            user_data = input("Enter the data you want to encrypt: ")
            
            encrypted, key = encrypt_data(user_data)
            store_data(identifier, encrypted, key)
            print(f"Data '{identifier}' encrypted and stored.\n")

        elif choice == 'D':
            identifier = input("Enter the identifier of the data you want to decrypt: ")
            decrypted_data = retrieve_and_decrypt(identifier)
            print(f"Decrypted data: {decrypted_data}\n")

        elif choice == 'Q':
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Please select 'E', 'D', or 'Q'.\n")

if __name__ == "__main__":
    main()


Data 'email' encrypted and stored.

Decrypted data: suraj@gmail.com

Data 'password' encrypted and stored.

Decrypted data: djtyfvkururuyiut7v5573w54we6gi

Goodbye!


In [3]:
from cryptography.fernet import Fernet

# Dictionary to store nodes in the hierarchical tree
tree_store = {}

# Function to encrypt data
def encrypt_data(data: str) -> (str, str):
    key = Fernet.generate_key()
    cipher_suite = Fernet(key)
    encrypted_data = cipher_suite.encrypt(data.encode())
    return encrypted_data.decode(), key.decode()

# Function to decrypt data
def decrypt_data(encrypted_data: str, key: str) -> str:
    cipher_suite = Fernet(key.encode())
    decrypted_data = cipher_suite.decrypt(encrypted_data.encode())
    return decrypted_data.decode()

# Function to create or update a node in the tree
def store_node(node_id: str, encrypted_data: str = None, key: str = None, child_keys: list = None):
    if encrypted_data is not None and key is not None:
        tree_store[node_id] = {'encrypted_data': encrypted_data, 'key': key, 'child_keys': None}
    else:
        tree_store[node_id] = {'encrypted_data': None, 'key': None, 'child_keys': child_keys}

# Function to retrieve and decrypt data from a leaf node
def retrieve_and_decrypt_leaf(node_id: str) -> str:
    if node_id in tree_store:
        node = tree_store[node_id]
        if node['encrypted_data'] and node['key']:
            return decrypt_data(node['encrypted_data'], node['key'])
        else:
            return "Node does not contain encrypted data."
    else:
        return "No such node found."

# Function to retrieve and decrypt data from an internal node using the aggregate key
def retrieve_and_decrypt_internal(node_id: str, master_key: str) -> dict:
    if node_id in tree_store:
        node = tree_store[node_id]
        if node['child_keys']:
            master_cipher_suite = Fernet(master_key.encode())
            decrypted_data = {}
            for child_node_id in node['child_keys']:
                child_node = tree_store[child_node_id]
                if child_node['key']:
                    child_key = master_cipher_suite.decrypt(child_node['key'].encode()).decode()
                    decrypted_data[child_node_id] = retrieve_and_decrypt_leaf(child_node_id)
                else:
                    decrypted_data[child_node_id] = "No key available for decryption."
            return decrypted_data
        else:
            return "No child nodes to decrypt."
    else:
        return "No such node found."

# Main function to handle user interaction
def main():
    while True:
        choice = input("Do you want to (E)ncrypt new data, (A)dd node, (D)ecrypt existing data, or (Q)uit? ").upper()
        
        if choice == 'E':
            identifier = input("Enter an identifier for the data (e.g., 'email', 'password'): ")
            user_data = input("Enter the data you want to encrypt: ")
            
            encrypted, key = encrypt_data(user_data)
            store_node(identifier, encrypted, key)
            print(f"Data '{identifier}' encrypted and stored.\n")

        elif choice == 'A':
            parent_id = input("Enter the parent node identifier: ")
            if parent_id in tree_store:
                node_id = input("Enter the new node identifier: ")
                if node_id not in tree_store:
                    # Generate an aggregate key for the parent node
                    aggregate_key = Fernet.generate_key()
                    child_keys = [parent_id]
                    store_node(node_id, child_keys=child_keys)
                    store_node(parent_id, key=aggregate_key.decode(), child_keys=[node_id])
                    print(f"Node '{node_id}' added under parent '{parent_id}'.\n")
                else:
                    print("Node identifier already exists.\n")
            else:
                print("Parent node not found.\n")

        elif choice == 'D':
            node_id = input("Enter the identifier of the node you want to decrypt: ")
            is_leaf = input("Is this a leaf node? (y/n): ").lower() == 'y'
            
            if is_leaf:
                decrypted_data = retrieve_and_decrypt_leaf(node_id)
                print(f"Decrypted data: {decrypted_data}\n")
            else:
                master_key = input("Enter the master key for the internal node: ")
                decrypted_data = retrieve_and_decrypt_internal(node_id, master_key)
                print(f"Decrypted data from child nodes: {decrypted_data}\n")

        elif choice == 'Q':
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Please select 'E', 'A', 'D', or 'Q'.\n")

if __name__ == "__main__":
    main()


Data 'email' encrypted and stored.

Parent node not found.

Parent node not found.

Goodbye!


In [6]:
from cryptography.fernet import Fernet

class HierarchicalKeyAggregateCryptosystem:
    def __init__(self):
        self.tree_store = {}  # Stores nodes in the hierarchical tree

    def generate_key(self) -> str:
        """Generate a new encryption key."""
        return Fernet.generate_key().decode()

    def encrypt_data(self, data: str, key: str) -> str:
        """Encrypt data using a given key."""
        cipher_suite = Fernet(key.encode())
        encrypted_data = cipher_suite.encrypt(data.encode())
        return encrypted_data.decode()

    def decrypt_data(self, encrypted_data: str, key: str) -> str:
        """Decrypt data using a given key."""
        cipher_suite = Fernet(key.encode())
        decrypted_data = cipher_suite.decrypt(encrypted_data.encode())
        return decrypted_data.decode()

    def store_node(self, node_id: str, encrypted_data: str = None, key: str = None, child_keys: list = None):
        """Store a node in the hierarchical tree."""
        self.tree_store[node_id] = {
            'encrypted_data': encrypted_data,
            'key': key,
            'child_keys': child_keys
        }

    def create_internal_node(self, parent_id: str, node_id: str):
        """Create an internal node that aggregates child keys."""
        if parent_id in self.tree_store:
            # Generate a new master key for the internal node
            master_key = self.generate_key()
            child_keys = self.tree_store[parent_id]['child_keys'][:]
            child_keys.append(node_id)

            # Encrypt the master key with the parent's key
            parent_key = self.tree_store[parent_id]['key']
            encrypted_master_key = self.encrypt_data(master_key, parent_key)
            
            # Store the internal node with the encrypted master key
            self.store_node(node_id, key=encrypted_master_key, child_keys=child_keys)
            
            # Update the parent node with the new child key (internal node)
            self.store_node(parent_id, key=parent_key, child_keys=child_keys)
            
            print(f"Internal node '{node_id}' added under parent '{parent_id}'.")
        else:
            print(f"Parent node '{parent_id}' not found.")

    def retrieve_and_decrypt_leaf(self, node_id: str) -> str:
        """Retrieve and decrypt data from a leaf node."""
        if node_id in self.tree_store:
            node = self.tree_store[node_id]
            if node['encrypted_data'] and node['key']:
                return self.decrypt_data(node['encrypted_data'], node['key'])
            else:
                return "Node does not contain encrypted data."
        else:
            return "No such node found."

    def retrieve_and_decrypt_internal(self, node_id: str) -> dict:
        """Retrieve and decrypt data from an internal node."""
        if node_id in self.tree_store:
            node = self.tree_store[node_id]
            if node['child_keys']:
                # Decrypt the master key for the internal node
                parent_key = self.tree_store[node_id]['key']
                master_key = self.decrypt_data(self.tree_store[node_id]['key'], parent_key)
                
                decrypted_data = {}
                for child_node_id in node['child_keys']:
                    child_node = self.tree_store[child_node_id]
                    if child_node['key']:
                        child_key = self.decrypt_data(child_node['key'], master_key)
                        decrypted_data[child_node_id] = self.retrieve_and_decrypt_leaf(child_node_id)
                    else:
                        decrypted_data[child_node_id] = "No key available for decryption."
                return decrypted_data
            else:
                return "No child nodes to decrypt."
        else:
            return "No such node found."

def main():
    hka_system = HierarchicalKeyAggregateCryptosystem()

    # Encrypt some data
    key1 = hka_system.generate_key()
    key2 = hka_system.generate_key()
    
    encrypted_data1 = hka_system.encrypt_data("Sensitive data 1", key1)
    encrypted_data2 = hka_system.encrypt_data("Sensitive data 2", key2)
    
    hka_system.store_node('leaf1', encrypted_data=encrypted_data1, key=key1)
    hka_system.store_node('leaf2', encrypted_data=encrypted_data2, key=key2)
    
    # Create an internal node
    hka_system.store_node('root', child_keys=['leaf1', 'leaf2'])
    hka_system.create_internal_node('root', 'internal_node')
    
    # Decrypt data using internal node
    decrypted_data = hka_system.retrieve_and_decrypt_internal('internal_node')
    print("Decrypted data from internal node:", decrypted_data)

if __name__ == "__main__":
    main()


AttributeError: 'NoneType' object has no attribute 'encode'